In [ ]:
import ee
import time
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

ee.Authenticate()
ee.Initialize(project='environment-change-detection')

zones = ee.FeatureCollection("projects/environment-change-detection/assets/MiningZones2")
features = zones.toList(zones.size())
num_zones = features.size().getInfo()

hansen = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")
loss_year = hansen.select("lossyear")
treecover = hansen.select("treecover2000").gt(30)

modis = ee.ImageCollection("MODIS/006/MCD12Q1") \
    .filterDate("2020-01-01", "2020-12-31") \
    .first() \
    .select("LC_Type1")
forest_mask = modis.remap([1, 2, 4, 5], [1, 1, 1, 1], 0)

DW = "GOOGLE/DYNAMICWORLD/V1"
geolocator = Nominatim(user_agent="mining_zone_locator")

def reverse_geocode(lat, lon, retries=3, timeout=5):
    for _ in range(retries):
        try:
            loc = geolocator.reverse((lat, lon), exactly_one=True, language='en', timeout=timeout)
            if loc:
                return loc.raw.get('display_name', '')
        except GeocoderTimedOut:
            time.sleep(2)
        except Exception:
            pass
    return None

results = []

for i in range(num_zones):
    try:
        zone = ee.Feature(features.get(i))
        name = zone.get("Name").getInfo().replace(" ", "_")
        lat = zone.get("Lat").getInfo()
        lon = zone.get("Long").getInfo()
        point = ee.Geometry.Point([lon, lat])
        region = ee.Geometry.Rectangle([lon - 0.025, lat - 0.025, lon + 0.025, lat + 0.025])

        loss_mask = loss_year.gte(20).And(loss_year.lte(24))
        forest_loss_pct = loss_mask.reduceRegion(
            ee.Reducer.mean(), region, 30, maxPixels=1e13
        ).get("lossyear").getInfo()

        dist_img = treecover.Not().fastDistanceTransform(30).sqrt()
        mean_distance = dist_img.reduceRegion(
            ee.Reducer.mean(), region, 30, maxPixels=1e13
        ).get("distance").getInfo()

        urban20 = ee.ImageCollection(DW).filterBounds(region).filterDate("2020-01-01", "2020-12-31") \
                                         .map(lambda img: img.select("label").eq(7)).mean()
        urban24 = ee.ImageCollection(DW).filterBounds(region).filterDate("2024-01-01", "2024-12-31") \
                                         .map(lambda img: img.select("label").eq(7)).mean()
        urban_growth_mean = urban24.subtract(urban20).reduceRegion(
            ee.Reducer.mean(), region, 10, maxPixels=1e13
        ).get("label").getInfo()

        lu20 = ee.ImageCollection(DW).filterBounds(region).filterDate("2020-01-01", "2020-12-31").select("label").mode()
        lu24 = ee.ImageCollection(DW).filterBounds(region).filterDate("2024-01-01", "2024-12-31").select("label").mode()
        lu_change_mean = lu24.subtract(lu20).abs().reduceRegion(
            ee.Reducer.mean(), region, 10, maxPixels=1e13
        ).get("label").getInfo()

        forest_vector = forest_mask.selfMask().reduceToVectors(
            geometry=point.buffer(50000),
            geometryType='centroid',
            scale=500,
            maxPixels=1e13
        )

        nearest = forest_vector.sort('system:distance').first()
        coords = nearest.geometry().coordinates().getInfo()
        forest_lon, forest_lat = coords

        forest_name = reverse_geocode(forest_lat, forest_lon)
        time.sleep(1.5)

        results.append({
            "Zone": name,
            "Lat": lat,
            "Long": lon,
            "ForestLossPct": forest_loss_pct,
            "DistanceToForest": mean_distance,
            "UrbanGrowth": urban_growth_mean,
            "LanduseChange": lu_change_mean,
            "ForestLat": forest_lat,
            "ForestLon": forest_lon,
            "ForestName": forest_name
        })

        print(f"✅ Processed: {name} → Forest: {forest_name}")
        time.sleep(3)

    except Exception as e:
        print(f"⚠️ Error zone {i}: {e}")

df = pd.DataFrame(results)
df.to_csv("zone_features_with_forest.csv", index=False)
df.head()

In [ ]:
import ee
import pandas as pd
import time

ee.Initialize(project='environment-change-detection')
df = pd.read_csv("zone_features_with_forest.csv")

def get_temperature(lat, lon, start="2023-01-01", end="2023-12-31"):
    point = ee.Geometry.Point([lon, lat])
    try:
        image = ee.ImageCollection("MODIS/006/MOD11A2") \
                    .filterDate(start, end) \
                    .select("LST_Day_1km") \
                    .mean() \
                    .multiply(0.02).subtract(273.15)
        result = image.reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()
        return round(result, 2) if result is not None else None
    except:
        return None

def get_NO2(lat, lon, start="2023-01-01", end="2023-12-31"):
    point = ee.Geometry.Point([lon, lat])
    try:
        image = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
                    .filterDate(start, end) \
                    .select("NO2_column_number_density") \
                    .mean()
        result = image.reduceRegion(ee.Reducer.mean(), point, 1000).get("NO2_column_number_density").getInfo()
        return round(result, 6) if result is not None else None
    except:
        return None

temp_mine_list = []
temp_forest_list = []
no2_mine_list = []
no2_forest_list = []

for i, row in df.iterrows():
    print(f"Processing zone {i+1}/{len(df)}: {row['Zone']}")
    lat_mine, lon_mine = row['Lat'], row['Long']
    lat_forest, lon_forest = row['ForestLat'], row['ForestLon']
    
    temp_mine_list.append(get_temperature(lat_mine, lon_mine))
    temp_forest_list.append(get_temperature(lat_forest, lon_forest))
    no2_mine_list.append(get_NO2(lat_mine, lon_mine))
    no2_forest_list.append(get_NO2(lat_forest, lon_forest))

    time.sleep(0.5)

df['Temp_Mine'] = temp_mine_list
df['Temp_Forest'] = temp_forest_list
df['NO2_Mine'] = no2_mine_list
df['NO2_Forest'] = no2_forest_list

df.to_csv("zone_features_enriched.csv", index=False)
print("✅ zone_features_enriched.csv saved with temp and NO₂")